In [1]:
import tensorflow as tf
import numpy as np
import yaml
from abc import abstractmethod

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
# module confEstimator
class TensorDict(dict):
    """
    Class that contains a dictionary of tensors
    """
    def last(self):
        return self.get(list(self.keys())[-1])
    
    def get_many(self, t_list):
        return {x: self.get(x) for x in t_list}
    
    def summary(self):
        print("** Specified Tensor Architecture **")
        for k, v in self.items():
            print("{}: {}".format(k, v))       
        print("**")
            
activation_dict = {"relu": tf.nn.relu, None: None, "None": None}

def create_conv_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    return {name: tf.layers.conv2d(inputs=tensor_in, 
                                   filters=spec['filters'],
                                   kernel_size=spec['kernel_size'],
                                   padding=spec['padding'],
                                   activation=activation_dict[spec['activation']],
                                   name=name)}

def create_maxpool_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    return {name: tf.layers.max_pooling2d(inputs=tensor_in,
                                          pool_size=spec['pool_size'],
                                          strides=spec['strides'],
                                          name=name)}

def create_dense_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    # creates an extra layer that must be flattened as well
    out = {}
    out[name+'_flatten'] = tf.reshape(tensor_in, [-1, np.prod(tensor_in.shape[1:4])])
    out[name] = tf.layers.dense(inputs=out[name+'_flatten'],
                                units=spec['units'],
                                activation=activation_dict[spec['activation']],
                                name=name)
    return out
                      
def create_dropout_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    return {name: tf.layers.dropout(inputs=tensor_in, rate=spec['rate'], training = mode == tf.estimator.ModeKeys.TRAIN) }

l_dict = {'CONV':create_conv_layer, 'POOL':create_maxpool_layer, 'FC': create_dense_layer, 'DROP': create_dropout_layer}

def tensorboard_summaries(conv_layers=None, dense_layers=None):
    if conv_layers is not None:
        for cl_name, cl in conv_layers.items():
            shape = cl.shape.as_list()
            for f in range(shape[3]):
                tf.summary.image("{}/filter/{}".format(cl_name, f), tf.reshape(cl[:, :, :, f], [tf.shape(cl)[0], shape[1], shape[2], 1]))  
    if dense_layers is not None:
        for dl_name, dl in dense_layers.items():
           tf.summary.histogram("{}/weights".format(dl_name), dl) 

        
class confEstimator(tf.estimator.Estimator):
    """
    Class that allows building a certain neural-net based on a configuration YAML file. It must be overcharged.
    """
            
    def __init__(self, conf_file, **kwargs):
        self.conf_file = conf_file
        with open(conf_file) as f:
            self.conf = yaml.load(f)
        super(confEstimator, self).__init__(self.conf_model_fn, **kwargs)
        
    @abstractmethod
    def conf_model_fn(self, features, labels, model):
        pass
    
    def _get_tensor_dict(self, data_in, mode):
        tensor_dict = TensorDict()  # will contain all the information on the layers and tensors
        # if data_in is a dictionary (e.g. 'x'), unroll
        if type(data_in)==dict:  # conver to np.array
            shaped_data_in = np.array([data_in[k] for k in data_in.keys()])
            if len(data_in.keys())==1:  # remove unncessary dimension
                shaped_data_in = shaped_data_in[0]
        else:
            shaped_data_in = data_in
        num_channels = 1 if len(shaped_data_in.shape)==3 else shaped_data_in.shape[3]
        tensor_dict['input_layer'] = tf.reshape(shaped_data_in, [-1, shaped_data_in.shape[1], shaped_data_in.shape[2], num_channels]) 
        for l_name, l_def in self.conf['layers'].items():
            tensor_dict.update(l_dict[l_def['type']](tensor_dict.last(), l_def['specs'], l_name, mode))
        return tensor_dict
    
    def get_layers_of_type(self, l_type):
        return [l_name for l_name, l_def in self.conf['layers'].items() if l_def['type']==l_type]
    

class confRegressorEstimator(confEstimator):
    """
    Class that allows for a image classification based on a convolutional neural-net which architecture is on a YAML file
    """
    def conf_model_fn(self, features, labels, mode):

        # tensor_dict
        td =  self._get_tensor_dict(features, mode)
        #td.summary()
        
        # tensorboard summary for weights and variables in the layers   
        tensorboard_summaries(conv_layers=td.get_many(self.get_layers_of_type('CONV')), 
                             dense_layers=td.get_many(self.get_layers_of_type('FC')))

        # predictions
        predictions = {
            "predictions": td.last()  # return the last predicted image
        }

        if mode == tf.estimator.ModeKeys.PREDICT:
            # wrap predictions into a class and return EstimatorSpec object
            return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
        
        # minimization function
        loss = tf.losses.mean_squared_error(labels=labels, predictions=td.last())  # loss is a scalar tensor
        
        if mode == tf.estimator.ModeKeys.EVAL:
            return tf.estimator.EstimatorSpec(mode=mode, loss=loss)

        if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.AdamOptimizer(learning_rate=self.conf['learning_rate'])
            train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)


        eval_metric_ops = {
            "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["predictions"]),
            "mean_absolute_error": tf.metrics.mean_absolute_error(labels=labels, predictions=predictions["predictions"]), 
            "RMS": tf.metrics.root_mean_squared_error(labels=labels, predictions=predictions["predictions"])
        }
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [3]:
import datetime
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt

sys.path.append("/jup/projects/mtgtools2/")  # ensure we find the MTG libraries
import pymtg.fci.esl.training_data as esl
import pymtg

RETINAS_JSON = Path(pymtg.__file__).parent.parent / "test/FCI_ESL/pyesl/config/retinas.json"
ESL_PATH = Path("/mnt/data/FCI/ESL/nc/output/20170615T152646/ch123/")
FCI_PATH = Path("/mnt/data/FCI/ESL/nc/swaths/")

def collapse_scene(scene, step=1):
    col_scene = scene[0]
    for s in scene[1:]:
        col_scene = np.insert(col_scene, -step, s.transpose()[-step], axis=1)  # create single input scene.
    return col_scene

def plot_scene(scene):
    plt.imshow(collapse_scene(scene))
    plt.title('Input scene')
    plt.show()

def plot_esl(esl):
    plt.imshow(esl)
    plt.title('ESL')
    plt.show()
    
def get_model_name(name='Unnamed_conf_model'):
    d = datetime.datetime.now()
    return "{}_{}".format(name, d.strftime("%Y%m%dT%H%M%S"))
                          
def get_model_dir(model_dir, **kwargs):
    return "{}{}".format(model_dir, get_model_name(**kwargs))
                          
    
def input_fn(size, train_or_test='train', collapsed=False, num_batches=1, mode=tf.estimator.ModeKeys.TRAIN, plot_data=False):
    files = esl.prepare_training_data(FCI_PATH, ESL_PATH, 0, str(RETINAS_JSON))
    out_batch = list(files[train_or_test].batches("ch123", size, 20000, num_batches=num_batches))  # use always 'train', 'test' is not validated
    if collapsed:
        features = np.array([collapse_scene(out_batch[i][0]) for i in range(num_batches)])
        labels = np.array([out_batch[i][1].flatten() for i in range(num_batches)])
    else:
        features = out_batch[0][0]  # only one batch
        labels = out_batch[0][1]    
    if plot_data:  # for validation purposes during prediction, will plot only the first image
        if collapsed:
            for f in features:
                plt.imshow(f)
                plt.show()
        else:
            plot_scene(features)
            plot_esl(out_batch[0][1])    
    dataset = tf.data.Dataset.from_tensors(({'features': features}, labels))
    return dataset

In [ ]:
from functools import partial

SIZE = 100
TRAIN_STEPS = 10
TEST_STEPS=10
MODEL = 'esl_cnn_collapsed'
NUM_BATCHES = 10
COLLAPSED = True

# config
run_config = tf.estimator.RunConfig(model_dir=get_model_dir("/mnt/data/tensorboard", name=MODEL),
                                    save_summary_steps=10, 
                                    save_checkpoints_steps=10)
                        
# create the estimator
esl_regressor = confRegressorEstimator('example_config.yml', config=run_config)
                                       
train_spec = tf.estimator.TrainSpec(input_fn=partial(input_fn, size=SIZE, num_batches=NUM_BATCHES, collapsed=COLLAPSED), 
                                    max_steps=TRAIN_STEPS)
eval_spec = tf.estimator.EvalSpec(input_fn=partial(input_fn, size=SIZE, num_batches=NUM_BATCHES, collapsed=COLLAPSED),
                                  steps=TEST_STEPS)

tf.estimator.train_and_evaluate(esl_regressor, train_spec, eval_spec)

INFO:tensorflow:Using config: {'_model_dir': '/mnt/data/tensorboardesl_cnn_collapsed_20180607T133932', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f185d4dc278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorf

In [ ]:
# single evaluation
foo = esl_regressor.evaluate(input_fn=partial(input_fn, size=TEST_SIZE), steps=1)

In [ ]:
foo = esl_regressor.predict(input_fn=partial(eval_input_fn, size=100, plot_data=True))
pred_esl = np.array([p['predictions'] for p in foo])

plt.imshow(pred_esl)
plt.title('Retrieved ESL')
plt.show()